# 数据探索模板

## 项目概述

**项目名称**: 估值预测项目
**创建日期**: {{date}}
**作者**: {{author}}

## 数据探索目标

1. 了解数据的基本结构和特征
2. 识别数据中的缺失值和异常值
3. 分析特征之间的相关性
4. 探索特征与目标变量的关系
5. 生成初步的数据可视化报告


In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import os

# 设置可视化风格
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# 加载配置文件
with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# 打印配置信息
print('配置文件加载成功!')
print(f'目标变量: {config['data']['target_column']}')
print(f'数据路径: {config['data']['raw_data_path']}')


## 1. 数据加载与基本信息

### 1.1 加载原始数据


In [ ]:
# 加载原始数据
data_path = config['data']['raw_data_path']
file_extension = config['data']['file_extension']

# 获取数据文件列表
data_files = [f for f in os.listdir(data_path) if f.endswith(file_extension)]
print(f'找到 {len(data_files)} 个数据文件: {data_files}')

# 加载第一个数据文件
if len(data_files) > 0:
    data = pd.read_csv(os.path.join(data_path, data_files[0]))
    print(f'数据加载成功，共 {data.shape[0]} 行，{data.shape[1]} 列')
else:
    print('未找到数据文件!')


In [ ]:
# 查看数据基本信息
print('
数据基本信息:')
data.info()

数据描述统计:')
data.describe()


In [ ]:
# 查看数据前5行
print('
数据前5行:')
data.head()


## 2. 数据质量分析

### 2.1 缺失值分析


In [ ]:
# 计算缺失值
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)) * 100

# 筛选出有缺失值的列
missing_data = pd.DataFrame({'缺失值数量': missing_values, '缺失值比例(%)': missing_percentage})
missing_data = missing_data[missing_data['缺失值数量'] > 0].sort_values('缺失值数量', ascending=False)

print('
缺失值分析:')
missing_data


In [ ]:
# 可视化缺失值分布
if len(missing_data) > 0:
    plt.figure(figsize=(12, 6))
    sns.barplot(x=missing_data.index, y=missing_data['缺失值比例(%)'])
    plt.title('各列缺失值比例')
    plt.xticks(rotation=45)
    plt.ylabel('缺失值比例(%)')
    plt.tight_layout()
    plt.show()


### 2.2 异常值分析


In [ ]:
# 选择数值列
numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
print(f'数值列数量: {len(numeric_columns)}')
print(f'数值列: {numeric_columns}')

# 可视化异常值（箱线图）
n_cols = 2
n_rows = (len(numeric_columns) + 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.boxplot(y=data[col], ax=axes[i])
    axes[i].set_title(f'{col} 箱线图')

# 隐藏多余的子图
for i in range(len(numeric_columns), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()


## 3. 特征相关性分析


In [ ]:
# 计算相关性矩阵
corr_matrix = data[numeric_columns].corr()

# 可视化相关性热力图
plt.figure(figsize=(15, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('特征相关性热力图')
plt.tight_layout()
plt.show()


In [ ]:
# 查看与目标变量相关性最高的特征
target_column = config['data']['target_column']
if target_column in corr_matrix.columns:
    target_corr = corr_matrix[target_column].sort_values(ascending=False)
    print(f'与 {target_column} 相关性最高的特征:')
    target_corr


## 4. 目标变量分析


In [ ]:
# 分析目标变量
if target_column in data.columns:
    plt.figure(figsize=(15, 6))
    
    # 直方图
    plt.subplot(1, 2, 1)
    sns.histplot(data[target_column], kde=True)
    plt.title(f'{target_column} 分布直方图')
    plt.xlabel(target_column)
    plt.ylabel('频率')
    
    # 箱线图
    plt.subplot(1, 2, 2)
    sns.boxplot(y=data[target_column])
    plt.title(f'{target_column} 箱线图')
    plt.ylabel(target_column)
    
    plt.tight_layout()
    plt.show()


## 5. 特征分布分析


In [ ]:
# 可视化数值特征分布
n_cols = 2
n_rows = (len(numeric_columns) + 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.histplot(data[col], kde=True, ax=axes[i])
    axes[i].set_title(f'{col} 分布')

# 隐藏多余的子图
for i in range(len(numeric_columns), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()


## 6. 数据探索总结

### 6.1 数据基本情况
- 数据规模: {{data.shape[0]}} 行，{{data.shape[1]}} 列
- 数值列数量: {{len(numeric_columns)}}
- 类别列数量: {{len(data.columns) - len(numeric_columns)}}

### 6.2 数据质量问题
- 缺失值情况: {{len(missing_data)}} 列存在缺失值
- 异常值情况: 部分数值列存在异常值

### 6.3 重要发现
1. 
2. 
3. 

### 6.4 建议后续处理
1. 缺失值处理：
2. 异常值处理：
3. 特征工程建议：
4. 模型选择建议：
